In [1]:
import findspark
import os
findspark.init('/usr/lib/apache_spark')

import pyspark
sc = pyspark.SparkContext()

In [5]:
from pathlib import Path
from zipfile import ZipFile
import requests
from math import ceil

taxidir = Path('taxidata')
if not taxidir.exists(): taxidir.mkdir(parents=True)

In [1]:
from ipywidgets import FloatProgress, HTML, VBox, HBox, FlexBox
from traitlets import Unicode
from datetime import datetime as DateTime

class TimedProgress(FlexBox):
    def __init__(self, *args, **kwargs):
        self._progress_bar = FloatProgress(*args, **kwargs)
        self._text = HTML()
        self._progresses = [HTML('') for _ in range(3)]
        self._time = HTML()
        self.start = DateTime.now()
        FlexBox.__init__(self, [self._text] + self._progresses + [self._time, self._progress_bar], orientation='vertical')
    
    @property
    def description(self):
        return self._text.value
    
    @description.setter
    def description(self, value):
        self._text.value = value
    
    @property
    def value(self):
        return self._progress_bar.value
    
    @staticmethod
    def interval_str(dt):
        days = dt.days
        hr, sec = divmod(dt.seconds, 3600)
        mn, sec = divmod(sec, 60)
        return (('%2dd ' % days) if days else '') + ('%2d:%02d:%02d' % (hr, mn, sec))
    
    @value.setter
    def value(self, val):
        now = DateTime.now()
        self._progress_bar.value = val
        min, max = self._progress_bar.min, self._progress_bar.max
        frac = (val - min)  / (max - min)
        if frac <= 0: frac = 1.0
        time_past = now - self.start
        total_time = time_past / frac
        eta = self.start + total_time
        left = total_time - time_past
        self._progresses[0].value = '%4s / %4s' % (val, max)
        self._progresses[1].value = 'Left: %s / %s' % (self.interval_str(left), self.interval_str(total_time))
        self._progresses[2].value = 'ETA: %s' % eta.strftime('%a %b %d, %H:%M:%S')

In [2]:
progress = TimedProgress(max=98)
display(progress)

for i in range(98):
    time.sleep(0.1)
    progress.value = i
    progress.description = 'fname'

NameError: name 'time' is not defined

In [3]:
from IPython.display import display
from ipywidgets import FloatProgress, HTML, VBox, HBox, FlexBox
import time
progress = FloatProgress(max=980, description='')
text1 = HTML()
text2 = HTML()
display(VBox([text1, text2, progress]))

for i in range(980):
    time.sleep(0.02)
    progress.value = i
    progress.max
    text1.value = 'fname: %4d / %4d' % (i, 980)

KeyboardInterrupt: 

In [6]:
txt = HTML()
display(txt)

for n in range(1, 13):
    trip_url = 'https://nyctaxitrips.blob.core.windows.net/data/trip_data_%d.csv.zip' % n
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    if path.exists():
        try:
            unzipped = ZipFile(str(path))
            continue
        except:
            txt.value = txt.value + ('Path %s found, but zipfile corrupt. Removing.\n' % path)
            path.unlink()
    
    request = requests.get(trip_url, stream=True)
    length = int(request.headers.get('Content-Length', 0))
    
    chunk_size = 1024*1024
    num_chunks = ceil(length / chunk_size)
    
    progress = TimedProgress(max=num_chunks)
    progress.description=str(path)
    display(progress)
    
    with path.open('wb') as fd:
        for chunk_num, chunk in enumerate(request.iter_content(chunk_size), 1):
            fd.write(chunk)
            progress.value = chunk_num
            progress.description = '%s' % str(path)

KeyboardInterrupt: 